## Introduction

In this notebook, we will solve for the vorticity evolution of the 2-d Navier-Stokes equation for viscous, incompressible fluid in vorticity form using a DeepONet. The branch consist of a UNet whose input is the initial condition, while the trunk's input are the spatial and temporal dimensions.

Data information can be obtained from https://github.com/oduolaidrisA/Scientific-Machine-Learning/blob/main/data_generation.ipynb. From the data, there are 100 time snapshots of the vorticity evolution in $[0,50]$. The spatial domain considered is $(0,1)^2$ with 256 steps each in the $x$ and $y$ axis. Feeding these in a trunk-net with a feed-forward neural network implies there are $256 \times 256 \times 100 = 6,553,600$ points to be fed into the trunk. This is extremely huge and will be computationally expensive to process.

A solution will be to use a separable deeponet $[1]$, where the trunk-net is a separable neural network. This basically means that instead of using a single feed-forward neural network in the trunk for the multi-dimensional coordinates, we can employ factorized coordinates and separate sub-networks for each on-dimensional domain. This means that for our problem the trunk consists of 3 sub-networks for the $x$, $y$ and $t$ domain. Thus, the total trunk input becomes $256+256+100 = 612$ points, which is ~$10,000 $ times decrease in the input. Thus, much more computationally efficient. It is important to note that this approach can only be utilized if the domain is separable, like the rectangular domain we have.

Another approach is to include the spatial coordinates as channels in the branch input. This approach was used in $[2]$, where they utilized DeepONet for C02 sequestration. This means that the trunk input now consist of only the time-domain $t$, so the input to the trunk in our case is just $100$ points, which is ~$65,000$ decrease from the initial input we had. However, this approach will only work if the domain is separable and the inputs and outputs of the deepONet are of the same dimensions.

For this notebook, we will utilize the second approach since we are solving for the vorticity evolution (which means the inputs and outputs are of same dimension).

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import seed_everything
from pathlib import Path
from sklearn.model_selection import train_test_split
torch.set_float32_matmul_precision('high') # or'high'. This is to properly utilize Tensor Cores of my CUDA device ('NVIDIA RTX A6000')
import h5py

seed_everything(42, workers=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Seed set to 42


cuda


In [ ]:
class config:
    def __init__(self):
        #The File paths
        self.data_path = 'C:/Users/idris_oduola/Documents/Projects/RqPINN/dataset/ns_data.h5' #Load data 
        self.model_path = 'C:/Users/idris_oduola/Documents/Projects/RqPINN/dataset/ns_model.pt'
        self.checkpoint_dir = 'C:/Users/idris_oduola/Documents/Projects/RqPINN/dataset/checkpoint_ns'


        #Model Parameters
        self.channelS = 50
        self.depth = 4
        self.base_channel = 64

        
        #Optimizer
        self.lr = 0.001
        self.weight_decay = 1e-4 #Regularization weight
        
        #The training parameters
        self.num_epoch = 100
        self.batch_size = 20

        #Learning rate scheduler
        self.step_size = 20  #To decay after every, say 10 epochs
        self.gamma = 0.5      #To reduce the learning rate by gamma (say, 1/2)

        

cfg = config()

#### Data Preparation

In [3]:
with h5py.File(cfg.data_path, 'r') as file:
    w_evolution = np.array(file['u'])

In [19]:
def input_dim(data):
    """
    A function to include the spatial dimensions

    """
    B,H,W,T = data.shape
    #The Spatial coordinates
    x_coords = torch.linspace(0,1, steps = W).view(1,-1).expand(H,W) #(256,256)
    y_coords = torch.linspace(0,1, steps = H).view(-1,1).expand(H,W) #(256,256)
    #Stacking the coordinates
    coord_stack = torch.tensor(np.stack([x_coords,y_coords], axis = -1)).float() #(256,256,2)
    coord_stack = coord_stack.unsqueeze(0).unsqueeze(3) # (1,256,256,1,2)
    coord_stack= coord_stack.repeat(B,1,1,T,1)
    #Now we concatenate the data with the information on the spatial dimension
    data1 = torch.tensor(data).unsqueeze(-1) # (B,256,256,T, 1)
    new_data = torch.cat([data1, coord_stack], dim = -1) # (B,256,256,T,3)
    assert new_data.shape[-1] == 3, print(f"Loaded input variables successfully. Data now has size {new_data.shape}")
    return new_data


In [24]:
w_evolution.shape

(500, 256, 256, 100)

In [25]:
def prepare_data(data,cfg):
    """
    Prepares the train, val and test data
    
    """
    new_data = input_dim(data)
    print(f"Target is of Shape:{data.shape}")

    #Branch input
    branch_input = new_data[:,:,:,0,:] #initial vorticity --> (B,256,256,3)
    print(f"Branch input is of Shape:{branch_input.shape}")
    training_points = data.shape[0]
    dataset = TensorDataset(branch_input, torch.tensor(data))
    train_set, val_set, test_set = random_split(dataset, [training_points - 100, 50, 50])

    #DataLoader
    train_loader = DataLoader(train_set, batch_size = cfg.batch_size, shuffle = True)
    val_loader = DataLoader(val_set, batch_size = cfg.batch_size, shuffle = False)
    test_loader = DataLoader(test_set, batch_size = cfg.batch_size, shuffle = False)

    return train_loader, val_loader, test_loader

In [ ]:
class DataModule(pl.LightningDataModule):
    def __init__(self,cfg):
        super().__init__()
        self.cfg = cfg
        self.train_loader = None
        self.val_loader = None
        self.test_loader = None

    def setup(self, stage = None):
        self.train_loader, self.val_loader, self.test_loader= prepare_data(w_evolution,self.cfg)
        print('DataLoaded Successfully!')

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.val_loader

    def test_dataloader(self):
        return self.test_loader

data_module = DataModule(cfg)
data_module.setup()

Target is of Shape:(500, 256, 256, 100)
Branch input is of Shape:torch.Size([500, 256, 256, 3])
DataLoaded Successfully


#### The DeepONet

> As indicated earlier, we will use a UNet in the branch. However, it is important to note that any suitable network can be used in the branch

##### The UNet
We adopt similar UNet structure as in 

In [27]:
#We will be defining the convolutional block
#This block will contain n_conv number of convolution layers, whose final layer downsamples the image resolution by 2 using strides
class conv_blocks(nn.Module):
    def __init__(self, in_channels, out_channels, stride, n_conv = 3, mid_channels = None):
        """
        n_conv*(conv --> batchnorm --> LeakyReLU)
        """
        super().__init__()
        mid_channels = mid_channels or out_channels
        def conv_block(in_ch, out_ch, strd):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=strd, padding = 1),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace = True)
            )
        layers = [conv_block(in_channels, mid_channels, 1)]
        layers += [conv_block(mid_channels, mid_channels, 1) for _ in range(n_conv -2)]
        layers += [conv_block(mid_channels, out_channels, stride)]
        self.block = nn.Sequential(*layers)
        
    def forward(self,x):
        return self.block(x)

### References

[1] Mandl, L., Goswami, S., Lambers, L., & Ricken, T. (2025). *Separable physics-informed DeepONet: Breaking the curse of dimensionality in physics-informed machine learning*. Computer Methods in Applied Mechanics and Engineering, 434, 117586.

[2] Diab, W., & Al Kobaisi, M. (2024). *U-DeepONet: U-Net enhanced deep operator network for geologic carbon sequestration*. Scientific Reports, 14(1), 21298.
